# Bloco 4 - Validação Cruzada

## Organizando e Reformulando os Dados

### Ideia

Eu vou dividir o dataframe entre os times. Então, para cada jogo, eu associarei a tendência de gols, escanteios, chutes, escanteios, entre outros, de cada time. Dessa forma, após a adição das tendências, eu terei um dataframe parecido com o inicial, porém, com uma adição de diversas novas features que tornarão o modelo de previsão mais completo.

In [28]:
# importanto algumas bibliotecas necessárias
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [29]:
# importanto o dataframe

# Código para ler o arquivo em xls e retorná-lo em dataframe
dfbr = pd.read_excel("dados brasileirao.xlsx")

# Nesta linha, foram tiradas todas as linhas que possuiam dados faltantes, visto que os anos iniciais não tinham todos os dados da tabela preenchidos.
dfbr = dfbr.dropna(axis = 0)

c:\venv\ilumpy\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


### Execução

In [30]:
# Essa função calcula, a partir de uma lista de valores, qual será a tendência dessa lista para o próximo valor.
def tendencia_de_uma_lista(lista_feature, razao):
    divisor = 1*(1 - razao**4)/(1 - razao) # normalização. Para quatro valores igual a n, a tendencia deverá ser n"
    soma = 0
    for i in range(4):
        termo = lista_feature[-(i+1)]*(razao**(i))
        soma = soma + termo
        
    tendencia = soma/divisor
    
    return round(tendencia, 3)

In [31]:
# função que, a partir de uma feature do mandante, retorna uma feature do visitante
def feature_conjugada(feature):
    if feature == 'gols_man':
        return 'gols_vis', 'time_man'
    
    elif feature == 'gols_vis':
        return 'gols_man', 'time_vis'
    
    elif feature == 'gols_1_tempo_man':
        return 'gols_1_tempo_vis','time_man'
    
    elif feature == 'gols_1_tempo_vis':
        return 'gols_1_tempo_man', 'time_vis'
    
    elif feature == 'escanteios_man':
        return 'escanteios_vis', 'time_man'
    
    elif feature == 'escanteios_vis':
        return 'escanteios_man', 'time_vis'
    
    elif feature == 'faltas_man':
        return 'faltas_vis', 'time_man'
    
    elif feature == 'faltas_vis':
        return 'faltas_man', 'time_vis'
        
    elif feature == 'chutes_bola_parada_man':
        return 'chutes_bola_parada_vis', 'time_man'
        
    elif feature == 'chutes_bola_parada_vis':
        return 'chutes_bola_parada_man', 'time_vis'
    
    elif feature == 'defesas_man':
        return 'defesas_vis', 'time_man'
    
    elif feature == 'defesas_vis':
        return 'defesas_man', 'time_vis'
    
    elif feature == 'impedimentos_man':
        return 'impedimentos_vis', 'time_man'
    
    elif feature == 'impedimentos_vis':
        return 'impedimentos_man', 'time_vis'
    
    elif feature == 'chutes_man':
        return 'chutes_vis', 'time_man'
    
    elif feature == 'chutes_vis':
        return 'chutes_man', 'time_vis'
        
    elif feature == 'chutes_fora_man':
        return 'chutes_fora_vis', 'time_man'
    
    elif feature == 'chutes_fora_vis':
        return 'chutes_fora_man', 'time_vis'
        
    else:
        print("Você tem certeza que essa feature faz parte do dataframe e que possui conjugado?")
        return False, False

In [32]:
# lista contendo as features alteradas
lista_das_features = ['gols_man', 'gols_1_tempo_man', 'escanteios_man', 'faltas_man', 'chutes_bola_parada_man',
                      'defesas_man', 'impedimentos_man', 'chutes_man', 'chutes_fora_man']

In [33]:
# função que cria uma lista das novas colunas de tendência

novas_colunas = []

for feat in lista_das_features:
    feat_conj = feature_conjugada(feat)[0]
    nome_nova_coluna = "tendencia_" + feat
    nome_nova_coluna_conj = "tendencia_" + feat_conj
    novas_colunas.append(nome_nova_coluna)
    novas_colunas.append(nome_nova_coluna_conj)

In [34]:
# função que adiciona as colunas de tendência ao dataframe

for coluna in novas_colunas:
    dfbr[coluna] = pd.NA

In [35]:
# função, em um determinado jogo, calcula as tendências de cada feature relativa ao mandante e ao visitante

def tendencia(index, razao):
    
    # Análise do time mandante
    time_man = dfbr.loc[index]["time_man"] # time mandante
    
    # Cria um dataframe contendo os jogos anteriores do time mandante
    df_anteriores_man = dfbr.loc[((dfbr["time_man"] == time_man) | (dfbr["time_vis"] == time_man)) & (dfbr.index < index)]
    
    # Análise do time visitante
    time_vis = dfbr.loc[index]["time_vis"] # time visitante
    
    # Cria um dataframe contendo os jogos anteriores do time visitante
    df_anteriores_vis = dfbr.loc[((dfbr["time_man"] == time_vis) | (dfbr["time_vis"] == time_vis)) & (dfbr.index < index)]
    
    # laço de repetição que modificará calculará cada tendência e a adicionará a colunas
    for feature in lista_das_features:
        
        # A partir do nome da feature do mandante, é encontrado o nome da feature relativo ao visitante
        feature_conj = feature_conjugada(feature)[0]
        
        # É a lista em que será baseada a tendencia do mandante
        lista_feature_time = []
        
        # Lista que seleciona associa os dados anteriores do time mandante
        for i in df_anteriores_man.index:
    
            if df_anteriores_man["time_man"][i] == time_man:
                lista_feature_time.append(df_anteriores_man[feature][i])
        
            if df_anteriores_man["time_vis"][i] == time_man:
                lista_feature_time.append(df_anteriores_man[feature_conj][i])
        
        # É a lista em que será baseada a tendencia do visitante
        lista_feature_conj_time = []
        
        # Lista que seleciona associa os dados anteriores do time visitante
        for j in df_anteriores_vis.index:

            if df_anteriores_vis["time_man"][j] == time_vis:
                lista_feature_conj_time.append(df_anteriores_vis[feature][j])
        
            if df_anteriores_vis["time_vis"][j] == time_vis:
                lista_feature_conj_time.append(df_anteriores_vis[feature_conj][j])
        
        # A partir da lista obtida, o código só prossegue se essa for maior que 4, o número de jogos analisados na tendência"
        if len(lista_feature_time) >= 4 and len(lista_feature_conj_time) >= 4:
            
            # utiliza a função tendencia_de_uma_lista para calcular a tendencia da feature do mandante
            tendencia_feature = tendencia_de_uma_lista(lista_feature_time, razao)
            
            # utiliza a função tendencia_de_uma_lista para calcular a tendencia da feature do visitante
            tendencia_feature_conj = tendencia_de_uma_lista(lista_feature_conj_time, razao)
            
            nome_coluna = "tendencia_" + feature
            
            nome_coluna_conj = "tendencia_" + feature_conj
            
            # adciona o resultado de tendência do mandante à coluna dessa tendência
            dfbr.at[index, nome_coluna] = tendencia_feature
            
            # adciona o resultado de tendência do mandante à coluna dessa tendência
            dfbr.at[index, nome_coluna_conj] = tendencia_feature_conj
            
    return 

In [36]:
# realiza a função tendencia para todas as linhas
for i in dfbr.index:
    tendencia(i, 0.5)

In [37]:
# Nos jogos em que os times não tinham um retrospecto de, pelo menos, quatro jogos para análise, as colunas de tendencia permaneceram vazias

# Nesta linha, novamente, foram tiradas todas as linhas que possuiam dados faltantes, visto que os anos iniciais não tinham todos os dados da tabela preenchidos.
dfbr = dfbr.dropna(axis = 0)

In [38]:
# Nesta linha, foram retiradas todas as colunas que possuiam dados relativos à estáticas após o começo do jogo. Os Dados após o jogo começar não nos ajudam a prever os nossos resultados, pois são muito específicos de cada jogo. Então vamos analisar o principal: número de gols
dfbr = dfbr.drop(["escanteios_man", "escanteios_vis", "publico_max", "faltas_man", "faltas_vis", "chutes_bola_parada_man",
                  "chutes_bola_parada_vis", "defesas_man", "defesas_vis", "impedimentos_man",
                  "impedimentos_vis", "chutes_man", "chutes_vis", "chutes_fora_man", "chutes_fora_vis",
                  "gols_1_tempo_man", "gols_1_tempo_vis", 'Unnamed: 0', 'ano_campeonato', 'data', 'horario',
                  'estadio', 'arbitro', 'time_man', 'time_vis', 'tecnico_man', 'tecnico_vis', 'rodada', 'publico'], axis=1)

In [39]:
dfbr = dfbr.reset_index(drop = True)
dfbr.head(5)

,colocacao_man,colocacao_vis,valor_equipe_titular_man,valor_equipe_titular_vis,idade_media_titular_man,idade_media_titular_vis,gols_man,gols_vis,tendencia_gols_man,tendencia_gols_vis,...,tendencia_chutes_bola_parada_man,tendencia_chutes_bola_parada_vis,tendencia_defesas_man,tendencia_defesas_vis,tendencia_impedimentos_man,tendencia_impedimentos_vis,tendencia_chutes_man,tendencia_chutes_vis,tendencia_chutes_fora_man,tendencia_chutes_fora_vis
0,17.0,9.0,6200000.0,9050000.0,22.2,31.0,1.0,0.0,0.533,1.6,...,17.333,9.933,4.667,4.067,1.333,0.733,3.267,2.8,4.867,5.6
1,5.0,14.0,47200000.0,10900000.0,30.3,24.5,4.0,0.0,1.2,1.133,...,15.2,13.733,4.4,2.4,1.533,2.533,4.0,6.667,2.133,5.867
2,12.0,11.0,11400000.0,35300000.0,27.8,26.3,0.0,3.0,1.733,2.067,...,20.067,18.067,2.867,2.6,0.067,3.267,5.667,4.067,4.733,3.067
3,8.0,6.0,17700000.0,18950000.0,26.3,29.6,1.0,1.0,0.6,0.6,...,14.267,17.267,5.0,3.667,1.133,2.467,3.067,4.533,2.733,10.467
4,3.0,19.0,41750000.0,6980000.0,28.8,28.9,0.0,1.0,1.4,0.267,...,15.533,14.533,1.733,2.8,0.467,1.6,6.0,2.733,4.933,3.4


## Validação Cruzada

### Regressões

#### Dados para regressões

In [40]:
# Divisão dos dados do dataset em treino e teste

from sklearn.model_selection import train_test_split # Divisão feita usando a função train_test_split

TAMANHO_TESTE = 0.1 # Porcentagem destinada do dataset ao grupo de teste
SEMENTE_ALEATORIA = 122106 # Definição de uma seed para evitar mudança desses grupos

# Escolha dos índices e seleção feita pela função
indices = dfbr.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

# Atribuição dos índices escolhidos aos dados
dfbr_treino = dfbr.loc[indices_treino].sort_index()
dfbr_teste = dfbr.loc[indices_teste].sort_index()

# Aqui são escolhidos os atributos do dataset e é definido o target 

FEATURES = ['colocacao_man', 'colocacao_vis', 'valor_equipe_titular_man',
       'valor_equipe_titular_vis', 'idade_media_titular_man',
       'idade_media_titular_vis', 'tendencia_gols_man',
       'tendencia_gols_vis', 'tendencia_gols_1_tempo_man',
       'tendencia_gols_1_tempo_vis', 'tendencia_escanteios_man',
       'tendencia_escanteios_vis', 'tendencia_faltas_man',
       'tendencia_faltas_vis', 'tendencia_chutes_bola_parada_man',
       'tendencia_chutes_bola_parada_vis', 'tendencia_defesas_man',
       'tendencia_defesas_vis', 'tendencia_impedimentos_man',
       'tendencia_impedimentos_vis', 'tendencia_chutes_man',
       'tendencia_chutes_vis', 'tendencia_chutes_fora_man',
       'tendencia_chutes_fora_vis']

TARGET = ["gols_man"]

# Divide os dados para se adequarem aos modelos

X_treino = dfbr_treino.reindex(FEATURES, axis=1).values
y_treino = dfbr_treino.reindex(TARGET, axis=1).values.ravel()
X_teste = dfbr_teste.reindex(FEATURES, axis=1).values
y_teste = dfbr_teste.reindex(TARGET, axis=1).values.ravel()

#### Regressão Linear

In [ ]:
"""Testando modelo classificatório de Regressão Linear com Validação Cruzada. 
Sabendo que o score nos dá a incerteza de um gol acontecer, vimos que esse modelo 
apresentou a incerteza de aproximadamente um gol e nos deu um resultado interessante 
comparado ao que fizemos nos blocos passados"""


from sklearn.linear_model import LinearRegression

# Cria um modelo linear
modelo_linear = LinearRegression()

# Treina esse modelo 
modelo_linear.fit(X_treino, y_treino)

scores = cross_val_score(
    modelo_linear,
    X_treino,
    y_treino,
    cv=NUM_FOLDS,
    scoring="neg_root_mean_squared_error" #rmse
)

print("Os scores foram de: ", scores)
print()
print("A média dos scores é de: ", scores.mean())

#### Árvore Aleatória 1 (Sem Hiperparâmetros)

In [ ]:
"""Usamos a árvore com validação cruzada e com o novo Dataframe, tivemos melhores resultados

# Árvore de Decisão

from sklearn.tree import DecisionTreeRegressor

# cria o modelo
modelo_dt = DecisionTreeRegressor(random_state=SEMENTE_ALEATORIA)

scores = cross_val_score(
    modelo_dt,
    X_treino,
    y_treino,
    cv=NUM_FOLDS,
    scoring="neg_root_mean_squared_error" #rmse
)

print("Os scores foram de: ", scores)
print()
print("A média dos scores é de: ", scores.mean())

#### Árvore Aleatória 2 (com Hiperparâmetros)

In [ ]:
# Árvore de decisões com hiperparâmetros

from itertools import product

NUM_FOLHAS = [None, 6, 36]
NUM_PROFUNDIDADE = [None, 1, 2, 3, 10]

for n_folhas, n_profundidade in product(NUM_FOLHAS, NUM_PROFUNDIDADE):
    modelo_dt = DecisionTreeRegressor(
        max_leaf_nodes=n_folhas,
        max_depth=n_profundidade,
        random_state=SEMENTE_ALEATORIA,
    )

    scores = cross_val_score(
        modelo_dt,
        X_treino,
        y_treino,
        cv=NUM_FOLDS,
        scoring="neg_root_mean_squared_error" #rmse
    )

    print("Os scores foram de: ", scores)
    print()
    print("A média dos scores é de: ", scores.mean())

In [ ]:
# Visualização da Árvore de Decisão

from matplotlib import pyplot as plt
from sklearn import tree

# Cria e treina o modelo
modelo_dt = DecisionTreeRegressor(
    max_leaf_nodes=6,
    random_state=SEMENTE_ALEATORIA,
)
modelo_dt.fit(X_treino, y_treino)

# Cria os objetos de figura e eixo
fig, axe = plt.subplots(
    ncols=1,
    nrows=1,
    figsize=(7, 4),
    dpi=150,
)

# Plota o grafo da árvore de decisão
tree.plot_tree(
    modelo_dt,
    feature_names=FEATURES,
    ax=axe,
    impurity=False,
    filled=True,
    proportion=True,
    precision=2,
)

# Salva a imagem
#plt.savefig('ÁrvoreDoFut.png',facecolor='white')

# Mostra o plot para o usuário
plt.show()

**Importância das Features** _(Assim como fizemos com o primeiro Dataframe para analisar a importância das features nas nossas decisões, vamos fazer com esse novo para comparar e discutir resultados)_

In [ ]:
N_ARVORES = 50
N_JOBS = 4

modelo_rf = RandomForestRegressor(
    n_estimators=N_ARVORES,
    random_state=SEMENTE_ALEATORIA,
    n_jobs=N_JOBS,
)

modelo_rf.fit(X_treino, y_treino)

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

importancia = modelo_rf.feature_importances_
desvio_padrao = np.std(
    [arvore.feature_importances_ for arvore in modelo_rf.estimators_], axis=0
)

serie_importancia = pd.Series(importancia, index=FEATURES)

fig, axe = plt.subplots()
serie_importancia.plot.bar(yerr=desvio_padrao, ax=axe)
axe.set_ylabel("Redução média da impureza")
plt.show()

#### Árvore de Aleatória 3 (com Hiperparâmetros)

In [ ]:
# Árvore de decisões com hiperparâmetros 2

from itertools import product

NUM_FOLHAS = [None, 5, 25]
NUM_PROFUNDIDADE = [None, 5, 10, 15, 20]

for n_folhas, n_profundidade in product(NUM_FOLHAS, NUM_PROFUNDIDADE):
    modelo_dt1 = DecisionTreeRegressor(
        max_leaf_nodes=n_folhas,
        max_depth=n_profundidade,
        random_state=SEMENTE_ALEATORIA,
    )

    scores = cross_val_score(
        modelo_dt1,
        X_treino,
        y_treino,
        cv=NUM_FOLDS,
        scoring="neg_root_mean_squared_error" #rmse
    )

print("Os scores foram de: ", scores)
print()
print("A média dos scores é de: ", scores.mean())

#### Floresta Aleatória 1

In [ ]:
NUM_ARVORES = 200
NUM_FOLDS = 13
NUM_CPU_CORES = 4

modelo_rf = RandomForestRegressor(
    n_estimators=NUM_ARVORES,
    random_state=SEMENTE_ALEATORIA,
    n_jobs=NUM_CPU_CORES,
   # max_leaf_nodes=3, max_depth=3,
)
scores = cross_val_score(
    modelo_rf,
    X_treino,
    y_treino,
    cv=NUM_FOLDS,
    scoring="neg_root_mean_squared_error", #RMSE
)

print("Os scores foram de: ", scores)
print()
print("A média dos scores é de: ", scores.mean())


Floresta mudando hiperparametros, com numero de árvores e de jobs

#### Floresta Aleatória 2

In [ ]:
NUM_ARVORES = 20
NUM_FOLDS = 13
NUM_CPU_CORES = 4

modelo_rf = RandomForestRegressor(
    n_estimators=NUM_ARVORES,
    random_state=SEMENTE_ALEATORIA,
    n_jobs=NUM_CPU_CORES,
    max_leaf_nodes=6, max_depth=6,
)
scores = cross_val_score(
    modelo_rf,
    X_treino,
    y_treino,
    cv=NUM_FOLDS,
    scoring="neg_root_mean_squared_error", #RMSE
)

print("Os scores foram de: ", scores)
print()
print("A média dos scores é de: ", scores.mean())

Árvore com novos hiperparametros apartir do anterior, alterando o numero máximo de folhas e suas profundidades 

### Classificações

#### Dados para classificações

In [41]:
import numpy as np

# Cria uma lista de condições
Condições = [
    (dfbr['gols_man'] > dfbr['gols_vis']),
    (dfbr['gols_man'] == dfbr['gols_vis']),
    (dfbr['gols_man'] < dfbr['gols_vis'])
    ]

# Cria uma lista com os possíveis resultados
Valores = ['Vitória', 'Empate', 'Derrota']

# Cria uma nova coluna com o resultado da partida para o mandante
dfbr['resultado_man'] = np.select(Condições, Valores)

dfbr.head()

,colocacao_man,colocacao_vis,valor_equipe_titular_man,valor_equipe_titular_vis,idade_media_titular_man,idade_media_titular_vis,gols_man,gols_vis,tendencia_gols_man,tendencia_gols_vis,...,tendencia_chutes_bola_parada_vis,tendencia_defesas_man,tendencia_defesas_vis,tendencia_impedimentos_man,tendencia_impedimentos_vis,tendencia_chutes_man,tendencia_chutes_vis,tendencia_chutes_fora_man,tendencia_chutes_fora_vis,resultado_man
0,17.0,9.0,6200000.0,9050000.0,22.2,31.0,1.0,0.0,0.533,1.6,...,9.933,4.667,4.067,1.333,0.733,3.267,2.8,4.867,5.6,Vitória
1,5.0,14.0,47200000.0,10900000.0,30.3,24.5,4.0,0.0,1.2,1.133,...,13.733,4.4,2.4,1.533,2.533,4.0,6.667,2.133,5.867,Vitória
2,12.0,11.0,11400000.0,35300000.0,27.8,26.3,0.0,3.0,1.733,2.067,...,18.067,2.867,2.6,0.067,3.267,5.667,4.067,4.733,3.067,Derrota
3,8.0,6.0,17700000.0,18950000.0,26.3,29.6,1.0,1.0,0.6,0.6,...,17.267,5.0,3.667,1.133,2.467,3.067,4.533,2.733,10.467,Empate
4,3.0,19.0,41750000.0,6980000.0,28.8,28.9,0.0,1.0,1.4,0.267,...,14.533,1.733,2.8,0.467,1.6,6.0,2.733,4.933,3.4,Derrota


In [42]:
# Divisão dos dados do dataset em treino e teste

from sklearn.model_selection import train_test_split # Divisão feita usando a função train_test_split

TAMANHO_TESTE = 0.1 # Porcentagem destinada do dataset ao grupo de teste
SEMENTE_ALEATORIA = 122106 # Definição de uma seed para evitar mudança desses grupos

# Escolha dos índices e seleção feita pela função
indices = dfbr.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

# Atribuição dos índices escolhidos aos dados
dfbr_treino = dfbr.loc[indices_treino].sort_index()
dfbr_teste = dfbr.loc[indices_teste].sort_index()

# Aqui são escolhidos os atributos do dataset e é definido o target 

FEATURES = ['colocacao_man', 'colocacao_vis', 'valor_equipe_titular_man',
       'valor_equipe_titular_vis', 'idade_media_titular_man',
       'idade_media_titular_vis', 'tendencia_gols_man',
       'tendencia_gols_vis', 'tendencia_gols_1_tempo_man',
       'tendencia_gols_1_tempo_vis', 'tendencia_escanteios_man',
       'tendencia_escanteios_vis', 'tendencia_faltas_man',
       'tendencia_faltas_vis', 'tendencia_chutes_bola_parada_man',
       'tendencia_chutes_bola_parada_vis', 'tendencia_defesas_man',
       'tendencia_defesas_vis', 'tendencia_impedimentos_man',
       'tendencia_impedimentos_vis', 'tendencia_chutes_man',
       'tendencia_chutes_vis', 'tendencia_chutes_fora_man',
       'tendencia_chutes_fora_vis']
TARGET = ["resultado_man"]

# Divide os dados para se adequarem aos modelos

X_treino = dfbr_treino.reindex(FEATURES, axis=1).values
y_treino = dfbr_treino.reindex(TARGET, axis=1).values.ravel()
X_teste = dfbr_teste.reindex(FEATURES, axis=1).values
y_teste = dfbr_teste.reindex(TARGET, axis=1).values.ravel()

#### K-nn / K-vizinhos

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

modelo_knn = KNeighborsClassifier()

scores = cross_val_score(
    modelo_knn,
    X_treino,
    y_treino,
    cv=10,
)


print("Os scores foram de: ", scores)
print()
print("A média dos scores é de: ", scores.mean())

In [ ]:
Deu certo o modelo_knn

#### Árvore Classificatória 1

#### Árvore Classificatória 2

#### Floresta Classificatória 1

In [ ]:
from sklearn.ensemble import RandomForestClassifier

modelo_crf = RandomForestClassifier( )
scores = cross_val_score(
    modelo_crf,
    X_treino,
    y_treino,
    cv=10,
)

print("Os scores foram de: ", scores)
print()
print("A média dos scores é de: ", scores.mean())

In [ ]:
Deu certo o modelo_crf

#### Floresta Classificatória 2

In [ ]:
from sklearn.ensemble import RandomForestClassifier

modelo_crf = RandomForestClassifier(random_state=SEMENTE_ALEATORIA,
                        n_estimators=100,  max_leaf_nodes=7, max_depth=7,          )
scores = cross_val_score(
    modelo_crf,
    X_treino,
    y_treino,
    cv=10,
)

print("Os scores foram de: ", scores)
print()
print("A média dos scores é de: ", scores.mean())

In [ ]:
Árvore com novos hiperparametros, classificação, alterando o numero máximo de folhas e suas profundidades.